# Amplifierd API - Module Management

This notebook demonstrates module discovery and source configuration.

## Overview

Modules in amplifier provide core functionality:
- **Providers**: LLM service integrations (OpenAI, Anthropic, etc.)
- **Tools**: Capabilities available to LLMs (file ops, web search, etc.)
- **Hooks**: Event-driven behaviors (logging, notifications, etc.)
- **Orchestrators**: Coordination and workflow management

This notebook covers both **discovery** (Phase 1) and **source management** (Phase 2).

In [ ]:
import json

import requests

BASE_URL = "http://127.0.0.1:8420"
API_BASE = f"{BASE_URL}/api/v1"


def print_response(response: requests.Response, title: str = "") -> None:
    if title:
        print(f"\n{'=' * 60}")
        print(f"{title}")
        print(f"{'=' * 60}")
    print(f"Status: {response.status_code} {response.reason}")
    if response.content:
        try:
            data = response.json()
            print(json.dumps(data, indent=2))
            return data
        except json.JSONDecodeError:
            print(response.text)
            return None
    return None


print("✓ Setup complete")

## Phase 1: Module Discovery

### List All Modules

Get all available modules across all types:

In [ ]:
response = requests.get(f"{API_BASE}/modules/")
modules = print_response(response, "LIST ALL MODULES")

if modules:
    print(f"\n✓ Found {len(modules)} module(s)")

    # Group by type
    by_type = {}
    for module in modules:
        module_type = module.get("type", "unknown")
        if module_type not in by_type:
            by_type[module_type] = []
        by_type[module_type].append(module)

    print("\nModules by type:")
    for module_type, modules_list in by_type.items():
        print(f"  - {module_type}: {len(modules_list)}")

### List Modules by Type

Get modules filtered by specific type:

In [ ]:
# List all providers
response = requests.get(f"{API_BASE}/modules/providers")
providers = print_response(response, "LIST PROVIDERS")

if providers:
    print(f"\n✓ Found {len(providers)} provider(s)")
    for provider in providers[:5]:  # Show first 5
        collection = provider.get("collection", "N/A")
        print(f"  - {provider['id']} (collection: {collection})")

In [ ]:
# List all tools
response = requests.get(f"{API_BASE}/modules/tools")
tools = print_response(response, "LIST TOOLS")

if tools:
    print(f"\n✓ Found {len(tools)} tool(s)")

In [ ]:
# List all hooks
response = requests.get(f"{API_BASE}/modules/hooks")
hooks = print_response(response, "LIST HOOKS")

if hooks:
    print(f"\n✓ Found {len(hooks)} hook(s)")

In [ ]:
# List all orchestrators
response = requests.get(f"{API_BASE}/modules/orchestrators")
orchestrators = print_response(response, "LIST ORCHESTRATORS")

if orchestrators:
    print(f"\n✓ Found {len(orchestrators)} orchestrator(s)")

### Get Module Details

Retrieve detailed information about a specific module:

In [ ]:
if providers:
    module_id = providers[0]["id"]

    response = requests.get(f"{API_BASE}/modules/{module_id}")
    details = print_response(response, f"GET MODULE: {module_id}")

    if details:
        print(f"\n✓ Module: {details['id']}")
        print(f"  Type: {details['type']}")
        print(f"  Location: {details['location']}")
        if details.get("collection"):
            print(f"  Collection: {details['collection']}")

### Filter Modules by Type

Use query parameters to filter modules:

In [ ]:
# Filter for providers only
response = requests.get(f"{API_BASE}/modules/?type=provider")
providers = print_response(response, "FILTER: type=provider")

if providers:
    print(f"\n✓ Found {len(providers)} provider module(s)")

## Phase 2: Module Source Management

### Add Module Source Override

Configure a custom source for a module:

**Note**: This modifies your local configuration.

In [ ]:
# Example: Add custom source for a module
module_source_data = {
    "source": "/path/to/custom/provider",
    "scope": "project",  # Options: user, project, local
}

module_id = "custom-provider"

# WARNING: This modifies your configuration
# Commented for safety

# response = requests.post(
#     f"{API_BASE}/modules/{module_id}/sources",
#     json=module_source_data
# )
# result = print_response(response, f"ADD SOURCE: {module_id}")
#
# if response.status_code == 201:
#     print(f"\n✓ Added source override:")
#     print(f"  Module: {result['module_id']}")
#     print(f"  Source: {result['source']}")
#     print(f"  Scope: {result['scope']}")

print("ℹ Add source example commented out for safety")
print("\nScope options:")
print("  - user: ~/.amplifier/settings.yaml")
print("  - project: .amplifier/settings.yaml")
print("  - local: .amplifier/settings.local.yaml")

### Update Module Source

Change the source for an existing module override:

In [ ]:
# Update module source
update_data = {"source": "/new/path/to/module", "scope": "project"}

module_id = "custom-provider"

# WARNING: This modifies your configuration
# response = requests.put(
#     f"{API_BASE}/modules/{module_id}/sources",
#     json=update_data
# )
# result = print_response(response, f"UPDATE SOURCE: {module_id}")
#
# if response.ok:
#     print(f"\n✓ Updated source to: {result['source']}")

print("ℹ Update source example commented out for safety")

### Remove Module Source Override

Delete a custom module source (reverts to default resolution):

In [ ]:
# Remove module source override
module_id = "custom-provider"
scope = "project"

# WARNING: This modifies your configuration
# response = requests.delete(
#     f"{API_BASE}/modules/{module_id}/sources?scope={scope}"
# )
# result = print_response(response, f"REMOVE SOURCE: {module_id}")
#
# if response.ok:
#     print(f"\n✓ Removed source override")
# elif response.status_code == 404:
#     print(f"\n✗ Source override not found")

print("ℹ Remove source example commented out for safety")

## Type-Specific Endpoints

Since module types are fundamental to amplifier core, the API provides type-specific endpoints for each module type. These delegate to the generic operations but provide clearer, more semantic URLs:

In [ ]:
# Type-specific endpoints for each module type

print("Type-Specific Endpoint Examples:")
print("=" * 60)

# Provider-specific
print("\nProviders:")
print("  POST   /api/v1/modules/providers/{id}/sources")
print("  PUT    /api/v1/modules/providers/{id}/sources")
print("  DELETE /api/v1/modules/providers/{id}/sources")

# Hook-specific
print("\nHooks:")
print("  POST   /api/v1/modules/hooks/{id}/sources")
print("  PUT    /api/v1/modules/hooks/{id}/sources")
print("  DELETE /api/v1/modules/hooks/{id}/sources")

# Tool-specific
print("\nTools:")
print("  POST   /api/v1/modules/tools/{id}/sources")
print("  PUT    /api/v1/modules/tools/{id}/sources")
print("  DELETE /api/v1/modules/tools/{id}/sources")

# Orchestrator-specific
print("\nOrchestrators:")
print("  POST   /api/v1/modules/orchestrators/{id}/sources")
print("  PUT    /api/v1/modules/orchestrators/{id}/sources")
print("  DELETE /api/v1/modules/orchestrators/{id}/sources")

print("\n" + "=" * 60)
print("✓ All type-specific endpoints delegate to generic operations")

### Example: Configure Provider Source

Using type-specific endpoint to configure a provider:

In [ ]:
# Example: Configure a custom provider using type-specific endpoint
provider_config = {"source": "/workspace/custom-openai-provider", "scope": "local"}

provider_id = "openai-provider"

# WARNING: This modifies your configuration
# Commented for safety

# response = requests.post(
#     f"{API_BASE}/modules/providers/{provider_id}/sources",
#     json=provider_config
# )
# result = print_response(response, f"CONFIGURE PROVIDER: {provider_id}")
#
# if response.status_code == 201:
#     print(f"\n✓ Provider source configured:")
#     print(f"  Module: {result['module_id']}")
#     print(f"  Source: {result['source']}")
#     print(f"  Scope: {result['scope']}")

print("ℹ Provider configuration example (commented for safety)")
print("\nType-specific endpoints provide clearer API semantics:")
print("  Generic:  POST /api/v1/modules/{id}/sources")
print("  Specific: POST /api/v1/modules/providers/{id}/sources")
print("\nBoth work identically - choose based on clarity preference")

### Type-Specific Write Operations

Configure module sources using type-specific endpoints:

#### Configure a Provider

In [ ]:
# Configure a custom provider source
provider_config = {"source": "/workspace/custom-provider", "scope": "local"}

provider_id = "openai-provider"

# WARNING: Modifies configuration - commented for safety
# response = requests.post(
#     f"{API_BASE}/modules/providers/{provider_id}/sources",
#     json=provider_config
# )
# result = print_response(response, f"CONFIGURE PROVIDER: {provider_id}")
#
# if response.status_code == 201:
#     print(f"\n✓ Provider configured: {result['module_id']}")
#     print(f"  Source: {result['source']}")
#     print(f"  Scope: {result['scope']}")

print("ℹ Provider configuration example (commented for safety)")
print(f"\nWould POST to: {API_BASE}/modules/providers/{provider_id}/sources")
print("With data:", json.dumps(provider_config, indent=2))

#### Configure a Hook

In [ ]:
# Configure a custom hook
hook_config = {"source": "git+https://github.com/team/custom-hook.git", "scope": "project"}

hook_id = "logging-hook"

# WARNING: Modifies configuration - commented for safety
# response = requests.post(
#     f"{API_BASE}/modules/hooks/{hook_id}/sources",
#     json=hook_config
# )
# result = print_response(response, f"CONFIGURE HOOK: {hook_id}")

print("ℹ Hook configuration example (commented for safety)")
print(f"\nWould POST to: {API_BASE}/modules/hooks/{hook_id}/sources")
print("With data:", json.dumps(hook_config, indent=2))

#### Configure a Tool

In [ ]:
# Configure a custom tool
tool_config = {"source": "/workspace/experimental-tool", "scope": "local"}

tool_id = "web-search-tool"

# WARNING: Modifies configuration - commented for safety
# response = requests.post(
#     f"{API_BASE}/modules/tools/{tool_id}/sources",
#     json=tool_config
# )
# result = print_response(response, f"CONFIGURE TOOL: {tool_id}")

print("ℹ Tool configuration example (commented for safety)")
print(f"\nWould POST to: {API_BASE}/modules/tools/{tool_id}/sources")
print("With data:", json.dumps(tool_config, indent=2))

#### Configure an Orchestrator

In [ ]:
# Configure a custom orchestrator
orchestrator_config = {"source": "~/.amplifier/modules/my-orchestrator", "scope": "user"}

orchestrator_id = "workflow-orchestrator"

# WARNING: Modifies configuration - commented for safety
# response = requests.post(
#     f"{API_BASE}/modules/orchestrators/{orchestrator_id}/sources",
#     json=orchestrator_config
# )
# result = print_response(response, f"CONFIGURE ORCHESTRATOR: {orchestrator_id}")

print("ℹ Orchestrator configuration example (commented for safety)")
print(f"\nWould POST to: {API_BASE}/modules/orchestrators/{orchestrator_id}/sources")
print("With data:", json.dumps(orchestrator_config, indent=2))

#### Update and Remove Type-Specific Sources

In [ ]:
# Update a provider source
update_config = {"source": "/updated/provider/path", "scope": "local"}

provider_id = "openai-provider"

# WARNING: Modifies configuration
# response = requests.put(
#     f"{API_BASE}/modules/providers/{provider_id}/sources",
#     json=update_config
# )
# result = print_response(response, f"UPDATE PROVIDER: {provider_id}")

print("ℹ Update provider source (commented for safety)")
print(f"Would PUT to: {API_BASE}/modules/providers/{provider_id}/sources")

print("\n" + "=" * 60)

# Remove a provider source
provider_id = "openai-provider"
scope = "local"

# WARNING: Modifies configuration
# response = requests.delete(
#     f"{API_BASE}/modules/providers/{provider_id}/sources?scope={scope}"
# )
# result = print_response(response, f"REMOVE PROVIDER SOURCE: {provider_id}")

print("\nℹ Remove provider source (commented for safety)")
print(f"Would DELETE: {API_BASE}/modules/providers/{provider_id}/sources?scope={scope}")

## Complete Module Discovery Workflow

Explore all modules across all types:

In [ ]:
def module_discovery_workflow():
    """Comprehensive module discovery workflow."""

    print("Module Discovery Workflow")
    print("=" * 60)

    module_types = ["providers", "tools", "hooks", "orchestrators"]

    for module_type in module_types:
        print(f"\n{module_type.upper()}:")
        response = requests.get(f"{API_BASE}/modules/{module_type}")

        if not response.ok:
            print(f"✗ Failed to list {module_type}")
            continue

        modules = response.json()
        print(f"Found {len(modules)} {module_type}")

        # Show details for first module
        if modules:
            module_id = modules[0]["id"]
            response = requests.get(f"{API_BASE}/modules/{module_id}")
            if response.ok:
                details = response.json()
                print(f"Example: {details['id']}")
                print(f"  Location: {details['location']}")
                if details.get("collection"):
                    print(f"  From collection: {details['collection']}")

    print("\n" + "=" * 60)
    print("✓ Discovery complete")


module_discovery_workflow()

## Practical Examples

### Finding Provider Modules

Discover which LLM providers are available:

In [ ]:
response = requests.get(f"{API_BASE}/modules/providers")
providers = print_response(response, "AVAILABLE PROVIDERS")

if providers:
    print("\nAvailable LLM Providers:")
    for provider in providers:
        source_info = f" (from {provider['collection']})" if provider.get("collection") else ""
        print(f"  - {provider['id']}{source_info}")

### Finding Tool Modules

Discover available tools:

In [ ]:
response = requests.get(f"{API_BASE}/modules/tools")
tools = print_response(response, "AVAILABLE TOOLS")

if tools:
    print("\nAvailable Tools:")
    for tool in tools:
        print(f"  - {tool['id']}")

### Understanding Module Sources

Modules can come from different sources:

In [ ]:
response = requests.get(f"{API_BASE}/modules/")
modules = response.json() if response.ok else []

if modules:
    print("Module Sources:")
    print("=" * 60)

    # Group by collection vs local
    from_collections = [m for m in modules if m.get("collection")]
    local_modules = [m for m in modules if not m.get("collection")]

    print(f"\nFrom Collections: {len(from_collections)}")
    for module in from_collections[:3]:
        print(f"  - {module['id']} (collection: {module['collection']})")

    print(f"\nLocal/Workspace: {len(local_modules)}")
    for module in local_modules[:3]:
        print(f"  - {module['id']}")

## Phase 2: Source Configuration Examples

### Use Case: Development Override

During development, you might want to use a local version of a module:

In [ ]:
print("Development Override Pattern:")
print("=" * 60)
print("\n1. Add local override for development:")
print("   POST /api/v1/modules/openai-provider/sources")
print("   {")
print('     "source": "/workspace/my-custom-openai-provider",')
print('     "scope": "local"  # Only for this machine')
print("   }")
print("\n2. Develop and test with your local version")
print("\n3. When done, remove override:")
print("   DELETE /api/v1/modules/openai-provider/sources?scope=local")
print("\n✓ Module resolution returns to default")

### Use Case: Team Configuration

Share module sources across your team:

In [ ]:
print("Team Configuration Pattern:")
print("=" * 60)
print("\n1. Add project-level source (checked into git):")
print("   POST /api/v1/modules/custom-tool/sources")
print("   {")
print('     "source": "git+https://github.com/team/custom-tool.git",')
print('     "scope": "project"  # Shared via .amplifier/settings.yaml')
print("   }")
print("\n2. Commit .amplifier/settings.yaml to version control")
print("\n3. Team members get the configuration automatically")
print("\n✓ Consistent module sources across team")

## Summary

### Phase 1 Operations (Read-Only)
- ✓ List all modules (with optional type filter)
- ✓ List modules by type (providers, tools, hooks, orchestrators)
- ✓ Get module details (location, type, collection)
- ✓ Discover module sources (collection vs local)

### Phase 2 Operations (Write)
- ✓ Add module source overrides (user/project/local scopes)
- ✓ Update existing source overrides
- ✓ Remove source overrides
- ✓ Scope-based configuration management
- ✓ Type-specific endpoints for semantic clarity

## API Endpoints Reference

### Generic Endpoints (Work for all types)
| Method | Endpoint | Description | Phase |
|--------|----------|-------------|-------|
| GET | `/api/v1/modules/` | List all modules | 1 |
| GET | `/api/v1/modules/?type={type}` | Filter by type | 1 |
| GET | `/api/v1/modules/{id}` | Get module details | 1 |
| POST | `/api/v1/modules/{id}/sources` | Add source override | 2 |
| PUT | `/api/v1/modules/{id}/sources` | Update source override | 2 |
| DELETE | `/api/v1/modules/{id}/sources` | Remove source override | 2 |

### Type-Specific Endpoints (Semantic wrappers)
| Method | Endpoint Pattern | Module Types |
|--------|------------------|--------------|
| GET | `/api/v1/modules/{type}` | providers, tools, hooks, orchestrators |
| POST | `/api/v1/modules/{type}/{id}/sources` | providers, tools, hooks, orchestrators |
| PUT | `/api/v1/modules/{type}/{id}/sources` | providers, tools, hooks, orchestrators |
| DELETE | `/api/v1/modules/{type}/{id}/sources` | providers, tools, hooks, orchestrators |

**Total**: 7 generic + 16 type-specific = **23 module endpoints**

## Configuration Files Modified

Write operations modify YAML files based on scope:
- `user`: `~/.amplifier/settings.yaml`
- `project`: `.amplifier/settings.yaml`
- `local`: `.amplifier/settings.local.yaml`

## Best Practices

1. **Use type-specific endpoints for clarity** - More semantic, easier to understand
2. **Use generic endpoints for automation** - Single code path handles all types
3. **Use LOCAL scope for development** - Machine-specific, not committed
4. **Use PROJECT scope for team standards** - Shared via git
5. **Use USER scope for personal defaults** - Applies to all your projects
6. **Test changes locally first** - Use local scope to verify before committing
7. **Document custom sources** - Add comments to settings.yaml explaining why

In [ ]:
print("Personal Defaults Pattern:")
print("=" * 60)
print("\n1. Add user-level source:")
print("   POST /api/v1/modules/preferred-provider/sources")
print("   {")
print('     "source": "~/.amplifier/modules/my-provider",')
print('     "scope": "user"  # Applies to all projects')
print("   }")
print("\n2. This setting applies across all your projects")
print("\n3. Project/local scopes can still override")
print("\n✓ Personal preferences respected everywhere")

## Configuration Scope Hierarchy

Understanding how scopes work:

In [ ]:
print("Configuration Scope Hierarchy (highest to lowest):")
print("=" * 60)
print("\n1. LOCAL (.amplifier/settings.local.yaml)")
print("   - Machine-specific overrides")
print("   - Not checked into version control")
print("   - Highest priority")
print("\n2. PROJECT (.amplifier/settings.yaml)")
print("   - Project-specific configuration")
print("   - Checked into version control")
print("   - Shared with team")
print("\n3. USER (~/.amplifier/settings.yaml)")
print("   - User-level defaults")
print("   - Applies to all projects")
print("   - Lowest priority")
print("\nExample:")
print("  USER scope: openai-provider -> /default/path")
print("  PROJECT scope: openai-provider -> /team/custom/path")
print("  LOCAL scope: openai-provider -> /my/dev/path")
print("  Result: Uses /my/dev/path (LOCAL wins)")

## Summary

### Phase 1 Operations (Read-Only)
- ✓ List all modules (with optional type filter)
- ✓ List modules by type (providers, tools, hooks, orchestrators)
- ✓ Get module details (location, type, collection)
- ✓ Discover module sources (collection vs local)

### Phase 2 Operations (Write)
- ✓ Add module source overrides (user/project/local scopes)
- ✓ Update existing source overrides
- ✓ Remove source overrides
- ✓ Scope-based configuration management

## API Endpoints Reference

| Method | Endpoint | Description | Phase |
|--------|----------|-------------|-------|
| GET | `/api/v1/modules/` | List all modules | 1 |
| GET | `/api/v1/modules/?type={type}` | Filter by type | 1 |
| GET | `/api/v1/modules/providers` | List providers | 1 |
| GET | `/api/v1/modules/tools` | List tools | 1 |
| GET | `/api/v1/modules/hooks` | List hooks | 1 |
| GET | `/api/v1/modules/orchestrators` | List orchestrators | 1 |
| GET | `/api/v1/modules/{id}` | Get module details | 1 |
| POST | `/api/v1/modules/{id}/sources` | Add source override | 2 |
| PUT | `/api/v1/modules/{id}/sources` | Update source override | 2 |
| DELETE | `/api/v1/modules/{id}/sources` | Remove source override | 2 |

## Configuration Files Modified

Write operations modify YAML files based on scope:
- `user`: `~/.amplifier/settings.yaml`
- `project`: `.amplifier/settings.yaml`
- `local`: `.amplifier/settings.local.yaml`

## Best Practices

1. **Use LOCAL scope for development** - Machine-specific, not committed
2. **Use PROJECT scope for team standards** - Shared via git
3. **Use USER scope for personal defaults** - Applies to all your projects
4. **Test changes locally first** - Use local scope to verify before committing
5. **Document custom sources** - Add comments to settings.yaml explaining why